# Assignment 3: All Pairs Similarity

## Setup & Dataset Retrieval

In [2]:
import os.path
import pickle
import random
from tqdm import tqdm

import src.utilities.io as io
import src.utilities.preprocess as pp
import src.apds.docsim_sequential as ds_seq
import src.apds.docsim_mapreduce_spark as ds_mr

/Users/a/miniforge3/envs/sbruch-assignment-Lwmd/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /Users/a/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/a/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
documents, data_path = io.get_data() #trec-covid as default

2023-05-31 18:15:36 - Loading Corpus...


  0%|          | 0/171332 [00:00<?, ?it/s]

2023-05-31 18:15:40 - Loaded 171332 TEST Documents.
2023-05-31 18:15:40 - Doc Example: {'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract 

filter out the documents with less than 10 words.

In [4]:
documents = {doc_id: doc for doc_id, doc in documents.items() if len(doc['text'].split()) >= 10 and doc['text'].strip() != ''}

In [5]:
for doc_id, doc in list(documents.items())[:10]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ug7v899j: 262 words
Document 02tnwd4m: 142 words
Document ejv2xln0: 219 words
Document 2b73a28n: 68 words
Document 9785vg6d: 110 words
Document zjufx4fo: 174 words
Document 5yhe786e: 113 words
Document 8zchiykl: 83 words
Document 8qnrcgnk: 188 words
Document jg13scgo: 189 words


### Sort the documents by their length (Longest to Shortest)

In [6]:
sort_docs = sorted(documents.items(), key=lambda x: len(x[1]['text']), reverse=True)
sorted_docs: pp.Documents = {}
for doc_id, doc in sort_docs:
    sorted_docs[doc_id] = doc

In [7]:
len(sorted_docs)

128592

In [8]:
for doc_id, doc in list(sorted_docs.items())[:50]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ij3ncdb6: 18000 words
Document c4pt07zk: 15448 words
Document 1vimqhdp: 12745 words
Document pd1g119c: 4642 words
Document gvh0wdxn: 3507 words
Document jkrj0lbm: 3783 words
Document jkvr9lq2: 3679 words
Document y00pz4fd: 1705 words
Document lqdgvsaq: 1627 words
Document vslevssr: 1504 words
Document o1vxgzpj: 1437 words
Document bt59txxp: 1474 words
Document yt9rj42s: 1460 words
Document qapwb5e4: 1529 words
Document rff3ikak: 1490 words
Document 0prx11md: 1310 words
Document 84hh0mg9: 1312 words
Document uumzl37q: 1053 words
Document vr7vm64u: 1097 words
Document v45cb2xq: 1136 words
Document yn7pu9i8: 1204 words
Document t473tbm9: 1271 words
Document nq0qe8qf: 1151 words
Document 16b8drw2: 1163 words
Document 4cbjbo7t: 1236 words
Document tvs1snq8: 1139 words
Document 2wb007gf: 1102 words
Document st1epx6u: 1114 words
Document syl7wq15: 1114 words
Document dt3li3bk: 1054 words
Document ec798c6s: 1057 words
Document xvnxkcxq: 1032 words
Document rxvxf34z: 1039 words
Documen

In [9]:
for doc_id, doc in list(sorted_docs.items())[-50:]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document denmp8yh: 10 words
Document 1k5h0tqi: 10 words
Document 46suh53t: 10 words
Document rmlklqwo: 15 words
Document vetdsk48: 10 words
Document h29uwe1d: 11 words
Document e4t19dza: 14 words
Document kg4oljan: 10 words
Document pfxyz3q3: 10 words
Document dvlwbho2: 11 words
Document fj9le8dx: 10 words
Document bg1amnoh: 11 words
Document 60p2a1i2: 10 words
Document suqvyeac: 13 words
Document h6r6qdna: 11 words
Document 6xqmlgr2: 10 words
Document igw288oj: 15 words
Document wt8ucsjr: 10 words
Document cjw88eoq: 11 words
Document n2v9kpm2: 12 words
Document srrdh09m: 12 words
Document l9n7mklv: 11 words
Document ob9i9t04: 11 words
Document 8j184di4: 11 words
Document yfs4kbus: 10 words
Document 5rmvdtrb: 12 words
Document kxbjtyjv: 11 words
Document pcl5bn54: 12 words
Document 08grs6m7: 10 words
Document sqos28f3: 12 words
Document isn31iqt: 12 words
Document 944w4qgq: 11 words
Document 1r60beaa: 10 words
Document 3nxp0j41: 10 words
Document i9aieai9: 11 words
Document i6b2bg6a: 1

This dictionary is used to map the index of a document to the corresponding text_id in the original dataset.
This index-to-ID mappings is useful for keeping track of the original text_id of the documents.

In [10]:
doc_idx_to_id = {i: doc_id for i, doc_id in enumerate(sorted_docs.keys())}

## Due to the size, we will use different dataset samples
(1000 longest docs, 1000 shortest docs, 1000 random docs)

### Define the paths to save and read the samples and their mappings

In [11]:
sample_size = 500
samples_dir = os.path.join(data_path, "samples_" + str(sample_size))
if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

random_docs_idx_path = os.path.join(samples_dir, "random_docs_idx.pkl")

In [12]:
if not os.path.exists(random_docs_idx_path):
    random_docs_idx = random.sample(range(len(sorted_docs)), k=sample_size)

    with open(random_docs_idx_path, "wb") as f:
        pickle.dump(random_docs_idx, f)
else:
    with open(random_docs_idx_path, "rb") as f:
        random_docs_idx = pickle.load(f)

longest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[:sample_size]:
    longest_docs[doc_id] = doc
longest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(sample_size))}

shortest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[-sample_size:]:
    shortest_docs[doc_id] = doc
shortest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(len(sorted_docs)-sample_size, len(sorted_docs)))}

random_docs = pp.Documents = {}
s_docs = list(sorted_docs.items())
for index in random_docs_idx:
    doc_id, doc = s_docs[index]
    random_docs[doc_id] = doc
random_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(random_docs_idx)}

Mapping is useful if we want to retrieve the original documents corresponding to the randomly chosen documents in random_docs

In [13]:
samples = [("longest_docs", longest_docs, longest_doc_idx_to_id),
           ("shortest_docs", shortest_docs, shortest_doc_idx_to_id),
           ("random_docs", random_docs, random_doc_idx_to_id)]

## Tokenization
Document cleaning and tokenization

In [14]:
tokens_dir = os.path.join(samples_dir, "tokens")
if not os.path.exists(tokens_dir):
    os.mkdir(tokens_dir)

docs_tokens_path = os.path.join(tokens_dir, "doc_tokens.pkl")

In [15]:
if not os.path.exists(docs_tokens_path):
    tokenized_samples = []
    for name, docs, idx_to_id in samples:
        tokenized_docs: pp.TokenizedDocuments = pp.get_tokenized_documents(docs)
        tokenized_samples.append((name, tokenized_docs, idx_to_id))

    with open(docs_tokens_path, "wb") as f:
        pickle.dump(tokenized_samples, f)
else:
    with open(docs_tokens_path, "rb") as f:
        tokenized_samples = pickle.load(f)

## VECTORIZE

In [16]:
tfidf_dir = os.path.join(samples_dir, "tfidf")
if not os.path.exists(tfidf_dir):
    os.mkdir(tfidf_dir)

tfidf_results_path = os.path.join(tfidf_dir, "tfidf_docs.pkl")

In [17]:
if not os.path.exists(tfidf_results_path):
    tfidf_docs = []
    for name, docs, idx_to_id in tqdm(tokenized_samples):
        vectorized_docs, time_taken = pp.vectorize(docs)
        tfidf_docs.append((name, vectorized_docs, time_taken, idx_to_id))

    with open(tfidf_results_path, "wb") as f:
        pickle.dump(tfidf_docs, f)
else:
    with open(tfidf_results_path, "rb") as f:
        tfidf_docs = pickle.load(f)

In [18]:
for name, _, time_taken, _ in tfidf_docs:
    print(f"Time taken to vectorize {name}: {time_taken:.6f} seconds")

Time taken to vectorize longest_docs: 0.591362 seconds
Time taken to vectorize shortest_docs: 0.024432 seconds
Time taken to vectorize random_docs: 0.119683 seconds


## Setup

In [19]:
thresholds = [0.3, 0.5, 0.7, 0.9] # Chosen thresholds

## Sequential APDS

In [20]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path = os.path.join(seq_results_dir, "seq_results.pkl")

In [21]:
#sample: pairs indices, pairs docs id, additional information
all_seq_results = ds_seq.exec_seq_apds(tfidf_docs=tfidf_docs,
                                       thresholds=thresholds,
                                       seq_results_path=seq_results_path,
                                       samples_dir=samples_dir,
                                       heuristic=False)

100%|██████████| 3/3 [00:03<00:00,  1.27s/it]


### longest 1000

In [22]:
ds_seq.print_results(all_seq_results, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 4682
tfidf_time: 0.5913619995117188
cosine_time: 0.1447899341583252
find_time: 0.18854117393493652
total_time: 0.9246931076049805

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 359
tfidf_time: 0.5913619995117188
cosine_time: 0.10183477401733398
find_time: 0.09501504898071289
total_time: 0.7882118225097656

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 209
tfidf_time: 0.5913619995117188
cosine_time: 0.16264009475708008
find_time: 0.11882591247558594
total_time: 0.8728280067443848

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 171
tfidf_time: 0.5913619995117188
cosine_time: 0.1150960922241211
find_time: 0.29085302352905273
total_time: 0.9973111152648926



In [23]:
ds_seq.print_first_pair(all_seq_results, "longest_docs", tokenized_samples)

1vimqhdp and pd1g119c have 0.5267 similarity

1vimqhdp:
36th international symposium intensive care emergency medicine brussels belgium 15-18 march 2016 p001 sepsis impairs capillary response within hypoxic capillary decrease erythrocyte oxygen-dependent atp efflux r. m. bateman m. d. sharpe j. e. jagger c. g. elli p002 lower serum immunoglobulin g2 level predispose severe flu j. solé-violán m. lópez-rodríguez e. herrera-ramos j. ruíz-hernández l. borderías j. horcajada n. gonzález-quevedo o. raja m. briones f. rodríguez de castro c. rodríguez gallego p003 brain protective effect intravenous immunoglobulin inhibition complement activation apoptosis rat model sepsis f. esen g. orhun p. ergin ozcan e. senturk c. ugur yilmaz n. orhan n. arican m. kaya m. kucukerden m. giris u. akcan s. bilgic gazioglu e. tuzun p004 adenosine a1 receptor dysfunction associated leukopenia possible mechanism sepsis-induced leukopenia r. riff o. naamani a. douvdevani p005 analysis neutrophil hyper spectral im

### shortest 1000

In [24]:
ds_seq.print_results(all_seq_results, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 596
tfidf_time: 0.02443218231201172
cosine_time: 0.013538122177124023
find_time: 0.14988112449645996
total_time: 0.1878514289855957

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 201
tfidf_time: 0.02443218231201172
cosine_time: 0.009026050567626953
find_time: 0.16397905349731445
total_time: 0.19743728637695312

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 120
tfidf_time: 0.02443218231201172
cosine_time: 0.05239391326904297
find_time: 0.35227394104003906
total_time: 0.42910003662109375

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 112
tfidf_time: 0.02443218231201172
cosine_time: 0.012727737426757812
find_time: 0.3026859760284424
total_time: 0.3398458957672119



In [25]:
ds_seq.print_first_pair(all_seq_results, "shortest_docs", tokenized_samples)

schpgyo0 and fenhvgs6 have 0.3332 similarity

schpgyo0:
covid-19 pneumonia patient without clear epidemiological history outside wuhan analysis radiographic clinical feature abstract purpose case report describe ct clinical feature covid-19 pneumonia patient without clear epidemiological history outside wuhan china

fenhvgs6:
bringing together in-depth look membrane fusion—a process essential communication within cells—is presented issue nature structural molecular biology

ehabkd78 and 5q5vwncj have 0.9063 similarity

ehabkd78:
covid-19 pandemic reshaping trade landscape covid-19 pandemic carry heavy threat preserving stable coordinated international trade relation essential avoid catastrophic disorder conflict

5q5vwncj:
’ next chapter present reflection heutagogy practice study conclusion already implemented others planned future

ehabkd78 and 5q5vwncj have 0.9063 similarity

ehabkd78:
covid-19 pandemic reshaping trade landscape covid-19 pandemic carry heavy threat preserving stable

### random 1000

In [26]:
ds_seq.print_results(all_seq_results, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 204
tfidf_time: 0.11968302726745605
cosine_time: 0.06319093704223633
find_time: 0.1497020721435547
total_time: 0.33257603645324707

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 13
tfidf_time: 0.11968302726745605
cosine_time: 0.04662823677062988
find_time: 0.4212496280670166
total_time: 0.5875608921051025

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 2
tfidf_time: 0.11968302726745605
cosine_time: 0.08374595642089844
find_time: 0.4139571189880371
total_time: 0.6173861026763916

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 2
tfidf_time: 0.11968302726745605
cosine_time: 0.05995488166809082
find_time: 0.2568817138671875
total_time: 0.4365196228027344



In [27]:
ds_seq.print_first_pair(all_seq_results, "random_docs", tokenized_samples)

4vbbxwd7 and du1dpb53 have 0.3320 similarity

4vbbxwd7:
sero-prevalence arthropod-borne viral infection among lukanga swamp resident zambia introduction re-emergence vector borne disease affecting million people recent year drawn attention arbovirus globally report sero-prevalence chikungunya virus chikv dengue virus denv mayaro virus mayv zika virus zikv swamp community zambia method collected blood saliva sample resident lukanga swamp 2016 mass-cholera vaccination campaign 10,000 resident vaccinated two dos shanchol 8482 period biological sample collected prior vaccination baseline specified time point vaccination tested total 214 baseline stored serum sample igg antibody ns1 denv zikv e2 chikv mayv elisa defined sero-prevalence proportion participant optical density od value defined cut-off value determined using finite mixture model result 214 participant 79 36.9 95 ci 30.5-43.8 sero-positive chikungunya 23 10.8 95 ci 6.9-15.7 zika 36 16.8 95 ci 12.1-22.5 dengue 42 19.6 95 ci 14.5-

## Sequential APDS with Heuristic

In [28]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path_h = os.path.join(seq_results_dir, "seq_results_h.pkl")

In [29]:
#sample: pairs indices, pairs docs id, additional information
all_seq_results_h = ds_seq.exec_seq_apds(tfidf_docs=tfidf_docs,
                                       thresholds=thresholds,
                                       seq_results_path=seq_results_path_h,
                                       samples_dir=samples_dir,
                                       heuristic=True)

100%|██████████| 3/3 [00:06<00:00,  2.09s/it]


### longest 1000

In [30]:
ds_seq.print_results(all_seq_results_h, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 4682
tfidf_time: 0.5913619995117188
cosine_time: 0.1587390899658203
find_time: 1.083571195602417
total_time: 1.833672285079956

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 359
tfidf_time: 0.5913619995117188
cosine_time: 0.11313724517822266
find_time: 0.373431921005249
total_time: 1.0779311656951904

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 209
tfidf_time: 0.5913619995117188
cosine_time: 0.09663009643554688
find_time: 0.3618600368499756
total_time: 1.0498521327972412

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 171
tfidf_time: 0.5913619995117188
cosine_time: 0.08811020851135254
find_time: 0.35250306129455566
total_time: 1.031975269317627



In [31]:
ds_seq.print_first_pair(all_seq_results_h, "longest_docs", tokenized_samples)

1vimqhdp and pd1g119c have 0.5267 similarity

1vimqhdp:
36th international symposium intensive care emergency medicine brussels belgium 15-18 march 2016 p001 sepsis impairs capillary response within hypoxic capillary decrease erythrocyte oxygen-dependent atp efflux r. m. bateman m. d. sharpe j. e. jagger c. g. elli p002 lower serum immunoglobulin g2 level predispose severe flu j. solé-violán m. lópez-rodríguez e. herrera-ramos j. ruíz-hernández l. borderías j. horcajada n. gonzález-quevedo o. raja m. briones f. rodríguez de castro c. rodríguez gallego p003 brain protective effect intravenous immunoglobulin inhibition complement activation apoptosis rat model sepsis f. esen g. orhun p. ergin ozcan e. senturk c. ugur yilmaz n. orhan n. arican m. kaya m. kucukerden m. giris u. akcan s. bilgic gazioglu e. tuzun p004 adenosine a1 receptor dysfunction associated leukopenia possible mechanism sepsis-induced leukopenia r. riff o. naamani a. douvdevani p005 analysis neutrophil hyper spectral im

### shortest 1000

In [32]:
ds_seq.print_results(all_seq_results_h, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 596
tfidf_time: 0.02443218231201172
cosine_time: 0.007924079895019531
find_time: 0.4426119327545166
total_time: 0.47496819496154785

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 201
tfidf_time: 0.02443218231201172
cosine_time: 0.01395416259765625
find_time: 0.786700963973999
total_time: 0.825087308883667

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 120
tfidf_time: 0.02443218231201172
cosine_time: 0.008003950119018555
find_time: 0.36437129974365234
total_time: 0.3968074321746826

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 112
tfidf_time: 0.02443218231201172
cosine_time: 0.008357048034667969
find_time: 0.3526132106781006
total_time: 0.3854024410247803



In [33]:
ds_seq.print_first_pair(all_seq_results_h, "shortest_docs", tokenized_samples)

schpgyo0 and fenhvgs6 have 0.3332 similarity

schpgyo0:
covid-19 pneumonia patient without clear epidemiological history outside wuhan analysis radiographic clinical feature abstract purpose case report describe ct clinical feature covid-19 pneumonia patient without clear epidemiological history outside wuhan china

fenhvgs6:
bringing together in-depth look membrane fusion—a process essential communication within cells—is presented issue nature structural molecular biology

ehabkd78 and 5q5vwncj have 0.9063 similarity

ehabkd78:
covid-19 pandemic reshaping trade landscape covid-19 pandemic carry heavy threat preserving stable coordinated international trade relation essential avoid catastrophic disorder conflict

5q5vwncj:
’ next chapter present reflection heutagogy practice study conclusion already implemented others planned future

ehabkd78 and 5q5vwncj have 0.9063 similarity

ehabkd78:
covid-19 pandemic reshaping trade landscape covid-19 pandemic carry heavy threat preserving stable

### random 1000

In [34]:
ds_seq.print_results(all_seq_results_h, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 204
tfidf_time: 0.11968302726745605
cosine_time: 0.03800320625305176
find_time: 0.38437819480895996
total_time: 0.5420644283294678

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 13
tfidf_time: 0.11968302726745605
cosine_time: 0.034245967864990234
find_time: 0.3610200881958008
total_time: 0.5149490833282471

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 2
tfidf_time: 0.11968302726745605
cosine_time: 0.03388094902038574
find_time: 0.36457300186157227
total_time: 0.5181369781494141

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 2
tfidf_time: 0.11968302726745605
cosine_time: 0.03452301025390625
find_time: 0.3743100166320801
total_time: 0.5285160541534424



In [35]:
ds_seq.print_first_pair(all_seq_results_h, "random_docs", tokenized_samples)

4vbbxwd7 and du1dpb53 have 0.3320 similarity

4vbbxwd7:
sero-prevalence arthropod-borne viral infection among lukanga swamp resident zambia introduction re-emergence vector borne disease affecting million people recent year drawn attention arbovirus globally report sero-prevalence chikungunya virus chikv dengue virus denv mayaro virus mayv zika virus zikv swamp community zambia method collected blood saliva sample resident lukanga swamp 2016 mass-cholera vaccination campaign 10,000 resident vaccinated two dos shanchol 8482 period biological sample collected prior vaccination baseline specified time point vaccination tested total 214 baseline stored serum sample igg antibody ns1 denv zikv e2 chikv mayv elisa defined sero-prevalence proportion participant optical density od value defined cut-off value determined using finite mixture model result 214 participant 79 36.9 95 ci 30.5-43.8 sero-positive chikungunya 23 10.8 95 ci 6.9-15.7 zika 36 16.8 95 ci 12.1-22.5 dengue 42 19.6 95 ci 14.5-

## Setup

In [39]:
thresholds = [0.3, 0.5, 0.7, 0.9]
# n_slices = [4, 7, 10]
n_executors = [4, 7, 9, 12]
n_slices = [7]

## Parallel APDS

In [40]:
mr_results_dir = os.path.join(samples_dir, "mr_result")
if not os.path.exists(mr_results_dir):
    os.mkdir(mr_results_dir)

mr_results_path = os.path.join(mr_results_dir, "mr_results.pkl")

In [42]:
# for collection_name, collection_docs, _ in samples:
all_mr_results = ds_mr.exec_mr_apds(tfidf_docs=tfidf_docs,
                                    thresholds=thresholds,
                                    n_executors=n_executors,
                                    n_slices=n_slices,
                                    mr_results_path=mr_results_path,
                                    samples_dir=samples_dir,
                                    heuristic=False)

  0%|          | 0/3 [00:00<?, ?it/s]


----working on longest_docs, threshold: 0.3----

----workers: 7, slices: 7----


2023-05-31 18:30:27.602 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext
2023-05-31 18:42:57.639 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on longest_docs, threshold: 0.5----

----workers: 7, slices: 7----


2023-05-31 18:52:19.249 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


...

----Done----



----working on random_docs, threshold: 0.7----

----workers: 7, slices: 7----


2023-05-31 19:11:30.521 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on random_docs, threshold: 0.9----

----workers: 7, slices: 7----


100%|██████████| 3/3 [41:52<00:00, 837.63s/it]                                  

----Done----




In [43]:
all_mr_results

{'longest_docs': {0.3: [([(506, 599, 0.33634986396979694),
     (491, 492, 1.0000000000000004),
     (461, 628, 0.3368609184342417),
     (66, 173, 0.3242321805217802),
     (205, 430, 0.41173450685400637),
     (28, 153, 0.34094847627745983),
     (246, 313, 0.3501735680440311),
     (376, 557, 0.3195987485436893),
     (400, 564, 0.3092593408210602),
     (570, 649, 0.40113235395114244),
     (537, 542, 0.33754091236715206),
     (40, 773, 0.3487890189910687),
     (163, 290, 0.3158022165334864),
     (281, 292, 0.3563162500413256),
     (167, 205, 0.3946405683718646),
     (258, 396, 0.3721242336141594),
     (244, 396, 0.35829232669767563),
     (25, 439, 0.3674239479718242),
     (52, 211, 0.35079743475758646),
     (64, 439, 0.4497539773144642),
     (176, 284, 0.33422516879382225),
     (120, 430, 0.3331251571369188),
     (117, 246, 0.4003965047877378),
     (138, 514, 0.34286736241936405),
     (430, 556, 0.5193877515413163),
     (542, 599, 0.5163667391702939),
     (158, 503

devo estrarre i dati in qualche modo

## Parallel APDS with Heuristic

In [46]:
mr_results_dir = os.path.join(samples_dir, "mr_result")
if not os.path.exists(mr_results_dir):
    os.mkdir(mr_results_dir)

mr_results_path_h = os.path.join(mr_results_dir, "mr_results_h.pkl")

In [47]:
# for collection_name, collection_docs, _ in samples:
all_mr_results_h = ds_mr.exec_mr_apds(tfidf_docs=tfidf_docs,
                                    thresholds=thresholds,
                                    n_executors=n_executors,
                                    n_slices=n_slices,
                                    mr_results_path=mr_results_path_h,
                                    samples_dir=samples_dir,
                                    heuristic=True)

  0%|          | 0/3 [00:00<?, ?it/s]2023-05-31 19:12:20.055 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext



----working on longest_docs, threshold: 0.3----

----workers: 7, slices: 7----


2023-05-31 19:24:32.377 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on longest_docs, threshold: 0.5----

----workers: 7, slices: 7----


2023-05-31 19:33:40.624 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


...

----Done----



----working on random_docs, threshold: 0.7----

----workers: 7, slices: 7----


2023-05-31 19:52:40.054 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----



----working on random_docs, threshold: 0.9----

----workers: 7, slices: 7----


100%|██████████| 3/3 [41:13<00:00, 824.63s/it]                                  

----Done----




In [48]:
all_mr_results_h

{'longest_docs': {0.3: [([(506, 599, 0.33634986396979694),
     (491, 492, 1.0000000000000004),
     (461, 628, 0.3368609184342417),
     (66, 173, 0.3242321805217802),
     (205, 430, 0.41173450685400637),
     (28, 153, 0.34094847627745983),
     (246, 313, 0.3501735680440311),
     (376, 557, 0.3195987485436893),
     (400, 564, 0.3092593408210602),
     (570, 649, 0.40113235395114244),
     (537, 542, 0.33754091236715206),
     (40, 773, 0.3487890189910687),
     (163, 290, 0.3158022165334864),
     (281, 292, 0.3563162500413256),
     (167, 205, 0.3946405683718646),
     (258, 396, 0.3721242336141594),
     (244, 396, 0.35829232669767563),
     (25, 439, 0.3674239479718242),
     (52, 211, 0.35079743475758646),
     (64, 439, 0.4497539773144642),
     (176, 284, 0.33422516879382225),
     (120, 430, 0.3331251571369188),
     (117, 246, 0.4003965047877378),
     (138, 514, 0.34286736241936405),
     (430, 556, 0.5193877515413163),
     (542, 599, 0.5163667391702939),
     (158, 503